# Manual Tuning Evaluation of Adapted CASTLE Models

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printe
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 

In [3]:
module_path = os.path.abspath(os.path.join('..'))
# Relative imports
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [12]:
from utils.setup import SetupDiagnostics
from neural_networks.load_models import load_models, get_save_plot_folder, load_single_model
from neural_networks.model_diagnostics import ModelDiagnostics
from pathlib import Path
from utils.variable import Variable_Lev_Metadata
import pickle
import itertools
import time
import datetime
import gc

In [6]:
def print_stats(stats):
    for stat, value in stats.items():
        print(f"{stat}: {value}")

In [7]:
def save_single_stats(stats, var, i_time, n_time, plot_dir):
    f_name = f"{i_time}_{n_time}_{var}.p"
    out_dir = os.path.join(plot_dir, "stats")
    Path(out_dir).mkdir(parents=True, exist_ok=True)
    
    with open(os.path.join(out_dir, f_name), "wb") as f:
        pickle.dump(stats, f)
    print(f"\nSaved stats {f_name}.")

In [8]:
def load_single_stats(var, i_time, n_time, plot_dir):
    f_name = f"{i_time}_{n_time}_{var}.p"
    with open(os.path.join(plot_dir, f_name), "rb") as f:
        var_stats = pickle.load(f)
    return var_stats

In [9]:
def compute_stats(md, var, plot_dir):
    i_time = "range" # this has to be range for it to work
    n_time = 1440
    diff = False

    md.compute_stats(i_time, var, nTime=n_time)
    print(f"\nStats for variable {var}: ")
    print_stats(md.stats)
    save_single_stats(md.stats, var, i_time, n_time, plot_dir)
    gc.collect()
    

In [10]:
def plot_lat_lon(md, var, plot_dir):
    i_time = "mean"
    diff = False
    n_time = 1440
    stats = ["mse", "r2"]
    
    plot_dir = os.path.join(plot_dir, "plots_lat_lon")
    
    md.plot_double_xy(i_time, var, diff=diff, nTime=n_time, stats=stats, 
                      cmap="RdBu_r", show_plot=False, save=plot_dir)
    gc.collect()

In [15]:
def run_eval_for_tuning_model(model_tuned):
    t_init_total = time.time()
    
    for p, s in itertools.product(lambda_prediction, lambda_sparsity):
        params_dir = f"lambda_pred_{p}-lambda_sparsity_{s}"

        log_dir = os.path.join(base_dir, model_tuned, params_dir)
        cfg = os.path.join(log_dir, config_name)
        output_dir = os.path.join(base_dir, "evaluation", model_tuned, params_dir)
        Path(output_dir).mkdir(parents=True, exist_ok=True)
        
        print(f"\n\n{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')} " + \
              f"--- Evaluating model {model_tuned} for parameters " + \
              f"lambda_prediciton={p}, lambda_sparsity={s}")
        t_init = time.time()
        setup = SetupDiagnostics(["-c", cfg])
        models = load_single_model(setup, var_name)

        model_key = setup.nn_type
        model_diagnostics = ModelDiagnostics(setup=setup, models=models) 

        variable = Variable_Lev_Metadata.parse_var_name(var_name)
        
        compute_stats(model_diagnostics, variable, output_dir)
        plot_lat_lon(model_diagnostics, variable, output_dir)
        
        t_end = datetime.timedelta(seconds=time.time() - t_init)
        
        print(f"\n{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')} " + \
              f"--- Finished for model {model_tuned} for parameters " + \
              f"lambda_prediciton={p}, lambda_sparsity={s}. Elapsed time: {t_end}")
        
        # There seems to be a memory issued, so let's try calling gc.collect()
        del models
        del model_diagnostics
        gc.collect()
        
    t_end_total = datetime.timedelta(seconds=time.time() - t_init_total)
    print(f"\n\n{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')} " + \
          f"--- Finished all evaluations for {model_tuned}. Total elapsed time: {t_end_total}\n\n")


In [16]:
# Set tuning parameters
lambda_prediction =[1, 5, 10, 100, 1000] #[1, 2, 4, 8, 10]
lambda_sparsity = [1.0] #[0.1, 0.5, 1.0]

# Set base directory for config files and inputs/outputs list files

# manual_tuning_tphystnd_691.39_v3-test2-transposed_l1_entry_scaled
# manual_tuning_tphystnd_691.39_v5-transposed_l1_entry_scaled_init_he
# manual_tuning_tphystnd_820.86_v3-transposed_l1_entry_scaled
# manual_tuning_tphystnd_820.86_v5-transposed_l1_entry_scaled_init_he


# manual_tuning_tphystnd_691.39_v3-test2-transposed_l1_matrix_scaled
# manual_tuning_tphystnd_691.39_v5-transposed_l1_matrix_scaled_init_he
# manual_tuning_tphystnd_820.86_v3-transposed_l1_matrix_scaled
# manual_tuning_tphystnd_820.86_v5-transposed_l1_matrix_scaled_init_he

PROJECT_ROOT = Path.cwd().parent.resolve()

base_dir = os.path.join(PROJECT_ROOT, "output_castle", "manual_tuning_tphystnd_820.86_v5-transposed_l1_entry_scaled_init_he") # "manual_tuning_tphystnd_691.39_v3-test2"
tuning_models = ["castle_adapted_big_notears"] 
                 # "castle_adapted_small_notears", "castle_adapted_small_dagma", "castle_adapted_big_dagma"] 
config_name = "cfg_castle_adapted.yml"

var_name = "tphystnd-820.86"# tphystnd-691.39, tphystnd-820.86



In [17]:
tunings = [#("manual_tuning_tphystnd_691.39_v3-test2-transposed_l1_matrix_scaled", "tphystnd-691.39"),
           #("manual_tuning_tphystnd_691.39_v5-transposed_l1_matrix_scaled_init_he", "tphystnd-691.39"), 
           ("manual_tuning_tphystnd_820.86_v3-transposed_l1_matrix_scaled", "tphystnd-820.86"),
           ("manual_tuning_tphystnd_820.86_v5-transposed_l1_matrix_scaled_init_he", "tphystnd-820.86")]

for tuning_dir, v in tunings:
    base_dir = os.path.join(PROJECT_ROOT, "output_castle", tuning_dir) 
    var_name = v
    
    t_start = time.time()
    
    print(f"\nBase directory: {base_dir}")
    for m in tuning_models:
        run_eval_for_tuning_model(m)

    t_finish =  datetime.timedelta(seconds=time.time() - t_start)

    print(f"\n\n{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')} " + \
          f"------ Finished. Total elapsed time: {t_finish}\n\n")


Base directory: /work/bd1179/b309247/pycharm_projects/iglesias-suarez2yxx_spuriouslinks/output_castle/manual_tuning_tphystnd_820.86_v3-transposed_l1_matrix_scaled


2023-11-30 09:17:50 --- Evaluating model castle_adapted_big_notears for parameters lambda_prediciton=1, lambda_sparsity=1.0

Load model: /work/bd1179/b309247/pycharm_projects/iglesias-suarez2yxx_spuriouslinks/output_castle/manual_tuning_tphystnd_820.86_v3-transposed_l1_matrix_scaled/castle_adapted_big_notears/lambda_pred_1-lambda_sparsity_1.0/models/CASTLEAdapted/r1.0-a1.0-lpred1.0-lspar1.0-lrec1.0-lacy1.0-NOTEARS-mirrored/hl_256_256_256_256_256_256_256_256_256-act_ReLU-e_18/1_22_model.keras

Computing horizontal stats for variable tphystnd-820.86

Test batch size = 8192.
Time samples: 1440

Stats for variable tphystnd-820.86: 
hor_tsqmean: 8.378718249991333e-09
hor_tmean: -2.7195295818880438e-06
hor_mse: 4.575346455937659e-09
hor_tvar: 8.371322408844569e-09
hor_r2: 0.4534499768992699

Saved stats range_1440_tphystnd-820.8

In [13]:
t_start = time.time()
for m in tuning_models:
    run_eval_for_tuning_model(m)

t_finish =  datetime.timedelta(seconds=time.time() - t_start)

print(f"\n\n{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')} " + \
      f"------ Finished. Total elapsed time: {t_finish}\n\n")



2023-11-29 14:30:17 --- Evaluating model castle_adapted_big_notears for parameters lambda_prediciton=1, lambda_sparsity=1.0

Load model: /work/bd1179/b309247/pycharm_projects/iglesias-suarez2yxx_spuriouslinks/output_castle/manual_tuning_tphystnd_820.86_v5-transposed_l1_entry_scaled_init_he/castle_adapted_big_notears/lambda_pred_1-lambda_sparsity_1.0/models/CASTLEAdapted/r1.0-a1.0-lpred1.0-lspar1.0-lrec1.0-lacy1.0-NOTEARS-mirrored/hl_256_256_256_256_256_256_256_256_256-act_ReLU-e_18/1_22_model.keras

Computing horizontal stats for variable tphystnd-820.86

Test batch size = 8192.
Time samples: 1440

Stats for variable tphystnd-820.86: 
hor_tsqmean: 8.378718249991333e-09
hor_tmean: -2.7195295818880438e-06
hor_mse: 4.669473477455921e-09
hor_tvar: 8.371322408844569e-09
hor_r2: 0.4422059921473729

Saved stats range_1440_tphystnd-820.86.p.

Plotting double_xy for variable tphystnd

Test batch size = 8192.
Time samples: 1440

Saved plot tphystnd-820.86_map_time-mean-1440steps.png.
Closed pl